In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fredapi import Fred
from datetime import datetime

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Configuration
# START_DATE = '2000-01-01'
START_DATE = '1990-01-01'  # earliest VIX data: gain ~2000 observations
# END_DATE = '2024-09-01'

# FRED series mapping
FRED_SERIES = {
    'VIX': 'VIXCLS',                    # VIX Volatility Index
    'Treasury_10Y': 'DGS10',            # 10-Year Treasury Rate
    'Yield_Spread': 'T10Y2Y',           # 10Y-2Y Yield Spread (daily, FRED-calculated, no lag adjustment needed)
    'CPI': 'CPIAUCSL',                  # Consumer Price Index
    'Unemployment': 'UNRATE',           # Unemployment Rate
    'Fed_Rate': 'FEDFUNDS',             # Federal Funds Rate
    'Consumer_Sentiment': 'UMCSENT',    # Consumer Sentiment Index
    # 'Credit_HY': 'BAMLH0A0HYM2',        # High yield credit spread
    # 'Credit_IG': 'BAMLC0A0CM'           # Investment grade credit spread
    # 'TB3MS': 'TB3MS',                   # for 10Y-3M yield spread - redundant with 10-2Y; removed
    # 'PCE': 'PCEPI',                     # alternative inflation measure - highly correlated with CPI; removed
    # 'GDP': 'GDP',                       # problematic. quarterly updates. vintage alignment complex; removed
    'Industrial_Production': 'INDPRO',  # Industrial Production Index
}

# Yahoo Finance tickers mapping
YAHOO_TICKERS = {
    # Volatility measures
    # 'VVIX': '^VVIX',                    # Volatility of VIX
    
    # Sector ETFs (Select Sector SPDRs)
    # 'Sector_Tech': 'XLK',               # Technology
    # 'Sector_Financials': 'XLF',         # Financials
    # 'Sector_Energy': 'XLE',             # Energy
    # 'Sector_Healthcare': 'XLV',         # Healthcare
    # 'Sector_ConsumerDiscretionary': 'XLY',  # Consumer Discretionary
    # 'Sector_ConsumerStaples': 'XLP',    # Consumer Staples
    # 'Sector_Industrials': 'XLI',        # Industrials
    # 'Sector_Materials': 'XLB',          # Materials
    # 'Sector_Utilities': 'XLU',          # Utilities
    # 'Sector_RealEstate': 'XLRE',        # Real Estate: began in 2015
    # 'Sector_Communications': 'XLC',     # Communication Services: began in 2018
    
    # Broad market measures
    'Wilshire5000': '^W5000',           # Wilshire 5000 Total Market Index
}

In [4]:
FRED_API_KEY = "c8d5b4c26407e7cbfcecca702e0e7aee"

In [5]:
# release date alignment method
USE_VINTAGE_DATES = False    # set True to use ALFRED, False for fixed shifts

In [6]:
def print_section(title, char='='):
    """print a formatted section header"""
    print(f'\n{char * 70}')
    print(f'{title}')
    print(f'{char * 70}\n')

def print_subsection(title):
    """print a formatted subsection header"""
    print(f'\n{title}')
    print(f'{"-" * 70}')

In [7]:
def collect_market_data(start_date=START_DATE,):
    """Collect S&P 500 market data from Yahoo Finance"""
    print_subsection('Collecting S&P 500 Market Data')

    # Download S&P 500 data
    sp500 = yf.download("^GSPC", start=start_date,  progress=False)

    # Handle MultiIndex columns if present
    if hasattr(sp500.columns, 'nlevels') and sp500.columns.nlevels > 1:
        sp500.columns = [col[0] for col in sp500.columns]

    # Calculate daily returns (percentage)
    sp500['Returns'] = sp500['Close'].pct_change() * 100

    # Keep only essential columns
    market_data = sp500[['Close', 'Volume', 'Returns']].copy()
    market_data.columns = ['SP500_Close', 'SP500_Volume', 'SP500_Returns']

    print(f'   Collected {len(market_data)} days of S&P 500 data')
    print(f'   Date range: {market_data.index[0]} to {market_data.index[-1]}')
    return market_data

In [8]:
def collect_fred_data(api_key, start_date=START_DATE):
    """Collect macroeconomic data from FRED"""
    print_subsection('Collecting FRED Macroeconomic Data')

    fred = Fred(api_key=api_key)
    fred_data = {}

    for name, series_id in FRED_SERIES.items():
        try:
            data = fred.get_series(series_id, start=start_date)
            fred_data[name] = data
            print(f"   ✓ {name}: {len(data)} observations")
        except Exception as e:
            print(f"   ✗ {name}: Error - {str(e)}")
            fred_data[name] = None

    print(f'\n   Summary: Successfully collected {sum(1 for v in fred_data.values() if v is not None)}/{len(FRED_SERIES)} series')
    return fred_data

In [9]:
def collect_yahoo_data(tickers, start_date=START_DATE):
    """
    collect data from Yahoo finance for multiple tickers
    
    params
    -----------
    tickers : dict
        dict mapping feature names toy ahoo ticker symbols
        example: {'VVIX': '^VVIX', 'Tech_Sector': 'XLK'}
    start_date : str
        start date for data collection
        
    retruns
    --------
    pd.DataFrame
        dataframe with all yahoo-sourced features, indexed by date
    """
    print_subsection('Collecting Yahoo Finance Data')
    
    yahoo_data = {}
    
    for name, ticker in tickers.items():
        try:
            data = yf.download(ticker, start=start_date, progress=False)
            
            # Handle MultiIndex columns if present
            if hasattr(data.columns, 'nlevels') and data.columns.nlevels > 1:
                data.columns = [col[0] for col in data.columns]
            
            # Keep only Close price (can add Volume later if needed)
            yahoo_data[name] = data['Close']
            print(f"   ✓ {name} ({ticker}): {len(data)} observations")
            
        except Exception as e:
            print(f"   ✗ {name} ({ticker}): Error - {str(e)}")
            yahoo_data[name] = None
    
    # Combine into single DataFrame, remove None entries
    yahoo_data = {k: v for k, v in yahoo_data.items() if v is not None}
    combined = pd.DataFrame(yahoo_data) if yahoo_data else pd.DataFrame()
    
    print(f'\n   Summary: Successfully collected {len(yahoo_data)}/{len(tickers)} tickers')
    return combined

In [10]:
def align_and_combine_data(market_data, fred_data, yahoo_data):
    """Align different frequency data and combine into master dataset"""
    print("\nAligning and combining data...")

    # Start with market data (daily frequency)
    master_df = market_data.copy()

    # Add FRED data (forward-fill for non-trading days)
    for name, series in fred_data.items():
        if series is not None:
            # Reindex to match market data dates and forward-fill
            aligned_series = series.reindex(master_df.index, method='ffill')
            master_df[name] = aligned_series

    # add yahoo data if provided
    if yahoo_data is not None and not yahoo_data.empty:
        for col in yahoo_data.columns:
            # Reindex to match market data dates and forward-fill
            aligned_series = yahoo_data[col].reindex(master_df.index, method='ffill')
            master_df[col] = aligned_series

    return master_df

In [11]:
def check_data_quality(df):
    '''
    Perform comprehensive data quality checks.
    '''
    print_subsection('Data Quality Check')
    
    # Check missing values
    total_missing = df.isnull().sum().sum()
    print(f'Total missing values: {total_missing}')
    if total_missing > 0:
        print('\nMissing by column:')
        missing_cols = df.isnull().sum()
        for col, count in missing_cols[missing_cols > 0].items():
            pct = (count / len(df)) * 100
            print(f'  {col}: {count} ({pct:.1f}%)')
    
    # Check for extreme returns (potential data errors)
    if 'SP500_Returns' in df.columns:
        extreme_returns = df[abs(df['SP500_Returns']) > 10]
        print(f'\nExtreme daily returns (>10%): {len(extreme_returns)}')
        if len(extreme_returns) > 0 and len(extreme_returns) < 10:
            print('Dates with extreme returns:')
            for date, ret in extreme_returns['SP500_Returns'].items():
                print(f'  {date.strftime("%Y-%m-%d")}: {ret:.2f}%')
    
    # Check data types
    print('\nData types:')
    print(df.dtypes)

In [12]:
def apply_vintage_date_alignment(df, api_key):
    '''
    apply exact release date alignment using ALFRED vintage dates
    
    falls back to forward-fill for periods where vintage data unavailable
    '''
    print_subsection('Applying ALFRED vintage date alignment')
    
    fred = Fred(api_key=api_key)
    df_aligned = df.copy()
    
    series_mapping = {
        'CPI': 'CPIAUCSL',
        # 'PCE': 'PCEPI',
        # 'TB3MS': 'TB3MS',
        'Unemployment': 'UNRATE',
        'Industrial_Production': 'INDPRO',
        'Fed_Rate': 'FEDFUNDS',
        'Consumer_Sentiment': 'UMCSENT'
    }
    
    alignment_report = []  # Track what happened to each series
    
    for col_name, series_id in series_mapping.items():
        print(f'  Processing {col_name} ({series_id})...')
        
        try:
            # Get all vintage releases
            vintages = fred.get_series_all_releases(series_id)
            vintages['date'] = pd.to_datetime(vintages['date'])
            vintages['realtime_start'] = pd.to_datetime(vintages['realtime_start'])
            
            # Create aligned series
            aligned_series = pd.Series(index=df_aligned.index, dtype=float)
            
            for idx in df_aligned.index:
                available_vintages = vintages[vintages['realtime_start'] <= idx]
                
                if len(available_vintages) > 0:
                    past_obs = available_vintages[available_vintages['date'] < idx]
                    
                    if len(past_obs) > 0:
                        most_recent = past_obs.sort_values(['date', 'realtime_start'], ascending=False).iloc[0]
                        aligned_series.loc[idx] = most_recent['value']
                    else:
                        aligned_series.loc[idx] = np.nan
                else:
                    aligned_series.loc[idx] = np.nan
            
            # Count vintage coverage
            vintage_count = aligned_series.notna().sum()
            total_count = len(df_aligned)
            
            # Store original for comparison
            original_series = aligned_series.copy()
            
            # If vintage has NaNs, fall back to original forward-filled values
            aligned_series = aligned_series.fillna(df_aligned[col_name])
            
            # Count how many were filled vs vintage
            filled_count = aligned_series.notna().sum() - vintage_count
            still_missing = aligned_series.isna().sum()
            
            # Replace column
            df_aligned[col_name] = aligned_series
            
            # Build report
            first_valid = aligned_series.first_valid_index()
            report = {
                'series': col_name,
                'vintage_values': vintage_count,
                'forward_filled': filled_count,
                'still_missing': still_missing,
                'first_valid_date': first_valid
            }
            alignment_report.append(report)
            
            # Print summary for this series
            if filled_count > 0 or still_missing > 0:
                print(f'    ✓ Aligned {col_name}: {vintage_count} vintage, {filled_count} forward-filled, {still_missing} missing')
                if first_valid:
                    print(f'      First valid date: {first_valid.strftime("%Y-%m-%d")}')
            else:
                print(f'    ✓ Aligned {col_name} ({vintage_count}/{total_count} all from vintage)')
        
        except Exception as e:
            print(f'    Error aligning {col_name}: {str(e)}')
            print(f'    Keeping original values')
            alignment_report.append({
                'series': col_name,
                'vintage_values': 0,
                'forward_filled': 0,
                'still_missing': len(df_aligned),
                'error': str(e)
            })
    
    # Print final summary
    print('\n VINTAGE ALIGNMENT SUMMARY:')
    print('─' * 70)
    for report in alignment_report:
        if 'error' in report:
            print(f"  {report['series']}: ERROR - {report['error']}")
        else:
            total = len(df_aligned)
            vintage_pct = (report['vintage_values'] / total * 100) if total > 0 else 0
            filled_pct = (report['forward_filled'] / total * 100) if total > 0 else 0
            print(f"  {report['series']}:")
            print(f"    - Vintage dates: {report['vintage_values']:,} ({vintage_pct:.1f}%)")
            if report['forward_filled'] > 0:
                print(f"    - Forward-filled: {report['forward_filled']:,} ({filled_pct:.1f}%)")
            if report['still_missing'] > 0:
                print(f"    - Still missing: {report['still_missing']:,}")
    
    return df_aligned

In [13]:
def apply_fixed_shift_alignment(df):
    '''
    apply fixed-day shift approximation for release dates.
    '''
    print_subsection('Applying fixed-shift release date alignment')
    
    # fix release date lags to prevent look-ahead bias
    # shift forward = make data available LATER (i.e. when actually released)
    df['CPI'] = df['CPI'].shift(14)  # released ~2 weeks after month end
    # df['PCE'] = df['PCE'].shift(21)  # released ~3 weeks after month end
    df['Unemployment'] = df['Unemployment'].shift(7)  # first fri of month
    df['Industrial_Production'] = df['Industrial_Production'].shift(14)  # ~2 weeks
    df['Fed_Rate'] = df['Fed_Rate'].shift(7)  # ~1 week after month end
    df['Consumer_Sentiment'] = df['Consumer_Sentiment'].shift(2)  # final release ~month end    
    # df['TB3MS'] = df['TB3MS'].shift(7)  # ~1 week after month end
    print('   ✓ Release date adjustments applied')
    
    return df

In [14]:
def preprocess_data(df):
    print("\nPreprocessing data...")

    # Print initial data quality
    check_data_quality(df)

    """
    print("\n=== CREDIT SPREAD MISSING DATA DIAGNOSTICS ===")
    if 'Credit_HY' in df.columns:
        missing_hy = df['Credit_HY'].isnull()
        missing_ig = df['Credit_IG'].isnull()
        
        print(f"Credit_HY missing: {missing_hy.sum()}")
        print(f"Credit_IG missing: {missing_ig.sum()}")
        
        if missing_hy.sum() > 0:
            print(f"First missing date: {df[missing_hy].index.min()}")
            print(f"Last missing date: {df[missing_hy].index.max()}")
            print(f"First valid date: {df[~missing_hy].index.min()}")
            
        # Check if missing values are contiguous (all at start/end)
        first_valid_idx = df['Credit_HY'].first_valid_index()
        last_valid_idx = df['Credit_HY'].last_valid_index()
        print(f"Credit spreads valid from {first_valid_idx} to {last_valid_idx}")

    print("\n=== MISSING DATA PATTERN ===")
    # Check how missing values are distributed
    missing_dates = df[df['Credit_HY'].isnull()].index
    missing_by_year = missing_dates.year.value_counts().sort_index()
    print("Missing Credit_HY observations by year:")
    print(missing_by_year)
    
    # Sample of dates with missing credit spreads
    print("\nSample of dates with missing credit spreads (first 20):")
    print(missing_dates[:20].tolist())
    
    # Check day of week pattern
    print("\nMissing by day of week:")
    print(missing_dates.dayofweek.value_counts().sort_index())
    print("(0=Monday, 6=Sunday)")
    """
    
    # forward-fill macro variables (monthly data in daily frequency)
    # market data (VIX, SP500, yields) should not have missing values
    print_subsection('Forward-filling macro variables...')
    macro_vars = ['CPI', 'Unemployment', 'Fed_Rate', 
                  'Consumer_Sentiment', 'Industrial_Production',
                   ]
    df_clean = df.copy()
    df_clean[macro_vars] = df_clean[macro_vars].fillna(method='ffill')
    
    # verify no remaining NaNs in critical columns
    remaining_na = df_clean[macro_vars].isnull().sum()
    if remaining_na.sum() > 0:
        print('   Warning: NaN values remain after forward-fill:')
        print(remaining_na[remaining_na > 0])
    else:
        print('   ✓ All macro variables forward-filled successfully')

    # apply release date alignment based on configuration
    if USE_VINTAGE_DATES:
        df_clean = apply_vintage_date_alignment(df_clean, FRED_API_KEY)
    else:
        df_clean = apply_fixed_shift_alignment(df_clean)
    
    # shifts will create NaNs that need to be removed
    df_clean = df_clean.dropna()

    if 'CPI' in df_clean.columns:
        df_clean['Inflation_YoY'] = df_clean['CPI'].pct_change(252) * 100  # 252 trading days ≈ 1 year
        # use 12 periods over 252 trading days to be more precise
        # df_clean['Inflation_YoY'] = df_clean['CPI'].pct_change(12) * 100

    # if 'PCE' in df_clean.columns:
    #     df_clean['PCE_YoY'] = df_clean['PCE'].pct_change(12) * 100

    if 'SP500_Close' in df_clean.columns:
        df_clean['SP500_Volatility'] = df_clean['SP500_Returns'].rolling(20).std()

    print(f"   ✓ Final dataset shape: {df_clean.shape}")
    print(f"   ✓ Date range: {df_clean.index[0]} to {df_clean.index[-1]}")
    return df_clean

In [15]:
def resample_to_frequency(df, freq='ME'):
    '''
    Resample daily data to weekly or monthly frequency.

    - takes last value for price levels
    - recalculates returns from resampled prices
    - for monthly: applies 1-month lag to macro variables
    '''
    print_subsection(f'\nResampling to {freq} frequency...')
    
    # Define aggregation rules
    # Special case: volume gets averaged, everything else gets end-of-period
    agg_rules = {}
    
    for col in df.columns:
        if 'Volume' in col:
            agg_rules[col] = 'mean'  # Average daily volume
        else:
            agg_rules[col] = 'last'  # End-of-period value for prices, rates, indices
    
    # Resample
    resampled = df.resample(freq).agg(agg_rules)
    
    # Recalculate derived features from resampled data
    if 'SP500_Close' in resampled.columns:
        resampled['SP500_Returns'] = resampled['SP500_Close'].pct_change() * 100
    if 'CPI' in resampled.columns:
        resampled['Inflation_YoY'] = resampled['CPI'].pct_change(12) * 100
    if 'SP500_Returns' in resampled.columns:
        resampled['SP500_Volatility'] = resampled['SP500_Returns'].rolling(12).std()
    
    # For monthly frequency: apply additional 1-period lag to macro variables
    if freq in ['M', 'ME', 'MS']:  # Month end, month start
        macro_vars = ['CPI', 'Unemployment', 'Industrial_Production', 
                      'Fed_Rate', 'Consumer_Sentiment']
        for var in macro_vars:
            if var in resampled.columns:
                resampled[var] = resampled[var].shift(1)
    
    # Drop NaNs from rolling windows and shifts
    resampled = resampled.dropna()
    
    print(f'   ✓ Resampled shape: {resampled.shape}')
    print(f'   ✓ Date range: {resampled.index[0]} to {resampled.index[-1]}')
    return resampled

In [16]:
def create_train_val_test_split(df, train_pct = 0.7, val_pct = 0.15):
    '''
    create temporal train/validation/test splits
    
    params:
    -------
    df : dataframe w/ DatetimeIndex
    train_pct : float, proportion for training (default 0.7)
    val_pct : float, proportion for validation (default 0.15)
    
    return:
    -------
    train, val, test : tuple of dataframes
    
    notes:
    -------
    uses temporal splits (not random) to avoid lookahead
    test set is most recent data
    '''
    n = len(df)
    train_end = int(n * train_pct)
    val_end = int(n * (train_pct + val_pct))
    
    train = df.iloc[:train_end].copy()
    val = df.iloc[train_end:val_end].copy()
    test = df.iloc[val_end:].copy()
    
    print('\n DATA SPLITS:')
    print('─' * 70)
    print(f'Train: {train.index[0].strftime("%Y-%m-%d")} to {train.index[-1].strftime("%Y-%m-%d")} ({len(train):,} obs, {train_pct*100:.0f}%)')
    print(f'Val:   {val.index[0].strftime("%Y-%m-%d")} to {val.index[-1].strftime("%Y-%m-%d")} ({len(val):,} obs, {val_pct*100:.0f}%)')
    print(f'Test:  {test.index[0].strftime("%Y-%m-%d")} to {test.index[-1].strftime("%Y-%m-%d")} ({len(test):,} obs, {(1-train_pct-val_pct)*100:.0f}%)')
    
    return train, val, test

In [17]:
def save_data(df, filename='data/financial_dataset.csv', label=''):
    """Save the processed dataset"""
    df.to_csv(filename)

    header = f' FINAL DATA SUMMARY - {label}' if label else ' FINAL DATA SUMMARY'
    print('\n' + '='*70)
    print(header)
    print('='*70)
    print(f'\nShape: {df.shape}')
    print(f'Date range: {df.index[0]} to {df.index[-1]}')
    print(f'Years covered: {(df.index[-1] - df.index[0]).days / 365.25:.1f}')
    print(f'\nColumns: {list(df.columns)}')
    print(f'\nMemory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB')
    print(f'\nTarget variable (SP500_Returns) statistics:')
    print(f'  Mean: {df["SP500_Returns"].mean():.4f}%')
    print(f'  Std: {df["SP500_Returns"].std():.4f}%')
    print(f'  Min: {df["SP500_Returns"].min():.4f}%')
    print(f'  Max: {df["SP500_Returns"].max():.4f}%')
    print(f'  Skewness: {df["SP500_Returns"].skew():.4f}')
    print(f'  Kurtosis: {df["SP500_Returns"].kurtosis():.4f}')
    print(f'\nMissing values: {df.isnull().sum().sum()}')

    if df.isnull().sum().sum() > 0:
        print('\nWARNING - Missing values by column:')
        for col in df.columns:
            missing = df[col].isnull().sum()
            if missing > 0:
                print(f'  {col}: {missing}')

    print(f"\nSaved dataset to '{filename}'")
    
    return filename

In [18]:
print_section('FINANCIAL DATA COLLECTION PIPELINE')

try:
    # Collect all data
    market_data = collect_market_data()
    fred_data = collect_fred_data(FRED_API_KEY)
    yahoo_data = collect_yahoo_data(YAHOO_TICKERS)

    # Combine and process
    combined_data = align_and_combine_data(market_data, fred_data, yahoo_data)
    final_data = preprocess_data(combined_data)

    # save daily version (raw)
    daily_filename = save_data(final_data, 'data/financial_dataset_daily.csv', 'DAILY')
    
    # create + save monthly version
    monthly_data = resample_to_frequency(final_data, freq='ME')
    monthly_filename = save_data(monthly_data, 'data/financial_dataset_monthly.csv', 'MONTHLY')

    print("\nData processing done and saved.")

except Exception as e:
    print(f"\nError during data collection: {str(e)}")
    print("Please check your FRED API key and internet connection.")


FINANCIAL DATA COLLECTION PIPELINE


----------------------------------------------------------------------
   Collected 9007 days of S&P 500 data
   Date range: 1990-01-02 00:00:00 to 2025-10-06 00:00:00

----------------------------------------------------------------------
   ✓ VIX: 9329 observations
   ✓ Treasury_10Y: 16634 observations
   ✓ Yield_Spread: 12875 observations
   ✓ CPI: 944 observations
   ✓ Unemployment: 932 observations
   ✓ Fed_Rate: 855 observations
   ✓ Consumer_Sentiment: 874 observations
   ✓ Industrial_Production: 1280 observations

   Summary: Successfully collected 8/8 series

----------------------------------------------------------------------
   ✓ Wilshire5000 (^W5000): 9000 observations

   Summary: Successfully collected 1/1 tickers

Aligning and combining data...

Preprocessing data...

Data Quality Check
----------------------------------------------------------------------
Total missing values: 150

Missing by column:
  SP500_Returns: 1 (0.0%)
  VI

In [20]:
print('\n Creating visualizations...')

fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 1. returns over time
axes[0].plot(final_data.index, final_data['SP500_Returns'], alpha=0.7, linewidth=0.5)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('S&P 500 Daily Returns Over Time', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Return (%)')
axes[0].grid(True, alpha=0.3)

# 2. distribution
axes[1].hist(final_data['SP500_Returns'].dropna(), bins=100, 
             edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_title('Distribution of Daily Returns', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Return (%)')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

# 3. cumulative returns
cumulative = (1 + final_data['SP500_Returns']/100).cumprod()
axes[2].plot(final_data.index, cumulative, linewidth=1.5)
axes[2].set_title('Cumulative Returns (Growth of $1)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Value ($)')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('reports/figures/data_overview.png', dpi=150, bbox_inches='tight')
print('   Saved visualization: data_overview.png')
plt.close()


 Creating visualizations...
   Saved visualization: data_overview.png


## Prepare Optimized data with Gold features

In [21]:
def calculate_rsi(series, window=14):
    """Calculate Relative Strength Index"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
def collect_gold_data_optimized(start_date, end_date):
    """Collect gold data with better error handling"""
    print("Collecting gold price data (optimized approach)...")

    try:
        # Try gold futures first
        print("   Attempting gold futures (GC=F)...")
        gold_data = yf.download('GC=F', 
                               start=start_date - pd.Timedelta(days=60), 
                               end=end_date + pd.Timedelta(days=1), 
                               progress=False)

        if hasattr(gold_data.columns, 'nlevels') and gold_data.columns.nlevels > 1:
            gold_data.columns = [col[0] for col in gold_data.columns]

        if len(gold_data) > 100:  # Reasonable amount of data
            print(f"   Gold futures data: {len(gold_data)} observations")
            return gold_data['Close']

    except Exception as e:
        print(f"   Gold futures failed: {e}")

    try:
        # Try SPDR Gold Trust ETF (GLD)
        print("   Attempting Gold ETF (GLD)...")
        gld_data = yf.download('GLD', 
                              start=start_date - pd.Timedelta(days=60), 
                              end=end_date + pd.Timedelta(days=1),
                              progress=False)

        if hasattr(gld_data.columns, 'nlevels') and gld_data.columns.nlevels > 1:
            gld_data.columns = [col[0] for col in gld_data.columns]

        if len(gld_data) > 100:
            # Convert GLD to approximate gold price (GLD ≈ gold/10)
            gold_proxy = gld_data['Close'] * 10
            print(f"   Gold ETF data: {len(gld_data)} observations")
            return gold_proxy

    except Exception as e:
        print(f"   Gold ETF failed: {e}")

    try:
        # Try iShares Gold Trust (IAU) 
        print("   Attempting Gold ETF (IAU)...")
        iau_data = yf.download('IAU', 
                              start=start_date - pd.Timedelta(days=60), 
                              end=end_date + pd.Timedelta(days=1),
                              progress=False)

        if hasattr(iau_data.columns, 'nlevels') and iau_data.columns.nlevels > 1:
            iau_data.columns = [col[0] for col in iau_data.columns]

        if len(iau_data) > 100:
            # Convert IAU to approximate gold price (IAU ≈ gold/100)
            gold_proxy = iau_data['Close'] * 100
            print(f"   Alternative Gold ETF data: {len(iau_data)} observations")
            return gold_proxy

    except Exception as e:
        print(f"   Alternative Gold ETF failed: {e}")

    print("BIG PROBLEM. GOLD DATA DOES NOT FOUND")

In [30]:
def add_optimized_gold_features(df):
    """
    Add ONLY the most valuable gold features that don't duplicate existing signals

    Selection criteria:
    1. Unique signal not captured by VIX/technical indicators
    2. Proven predictive value in research
    3. Low correlation with existing features
    4. Captures different market dynamics (monetary policy, crisis, etc.)
    """
    print(" Adding optimized gold features (quality over quantity)...")

    # Get gold data
    start_date = df.index[0]
    end_date = df.index[-1]
    
    gold_prices = collect_gold_data_optimized(start_date, end_date)

    # Align with stock data
    gold_prices_aligned = gold_prices.reindex(df.index, method='ffill')

    # Core gold features - these provide unique signals
    df['Gold_Price'] = gold_prices_aligned
    df['Gold_Returns'] = gold_prices_aligned.pct_change() * 100

    print("   ✓ Basic gold price and returns")

    # 1. GOLD/SPX RATIO - Key risk-on/risk-off indicator (UNIQUE SIGNAL)
    if 'SP500_Close' in df.columns:
        df['Gold_SPX_Ratio'] = df['Gold_Price'] / df['SP500_Close']
        # Normalized version to capture deviations
        df['Gold_SPX_Ratio_Norm'] = ((df['Gold_SPX_Ratio'] - df['Gold_SPX_Ratio'].rolling(60).mean()) / 
                                     df['Gold_SPX_Ratio'].rolling(60).std())
        print("   ✓ Gold/SPX ratio (primary risk sentiment indicator)")

    # 2. GOLD vs REAL INTEREST RATES - Monetary policy impact (UNIQUE SIGNAL)
    if 'Treasury_10Y' in df.columns:
        # Approximate real rates using Treasury yield minus rolling inflation proxy
        if 'CPI' in df.columns:
            inflation_proxy = df['CPI'].pct_change(252) * 100  # YoY change
            df['Real_Interest_Rate'] = df['Treasury_10Y'] - inflation_proxy
            df['Gold_Real_Rate_Signal'] = (df['Real_Interest_Rate'] < 0).astype(int)
            print("   ✓ Gold vs real interest rates (monetary policy signal)")
        else:
            # Simplified version without CPI
            df['Gold_Yield_Inverse'] = 1 / (df['Treasury_10Y'] + 0.01)  # Avoid division by zero
            print("   ✓ Gold vs nominal rates (simplified monetary signal)")

    # 3. GOLD MOMENTUM - Different timeframe than stock momentum (COMPLEMENTARY SIGNAL)
    df['Gold_Momentum_10d'] = df['Gold_Price'].pct_change(10)
    df['Gold_Momentum_Strength'] = (df['Gold_Momentum_10d'] > df['Gold_Momentum_10d'].rolling(60).quantile(0.7)).astype(int)
    print("   ✓ Gold momentum (10-day, different from stock momentum)")

    # 4. GOLD VOLATILITY REGIME - Crisis detection (UNIQUE SIGNAL)
    gold_vol = df['Gold_Returns'].rolling(20).std()
    df['Gold_Vol_Regime'] = (gold_vol > gold_vol.rolling(120).quantile(0.8)).astype(int)
    print("   ✓ Gold volatility regime (crisis detection)")

    # 5. GOLD TREND PERSISTENCE - Long-term trend strength (UNIQUE SIGNAL)
    gold_ma_short = df['Gold_Price'].rolling(20).mean()
    gold_ma_long = df['Gold_Price'].rolling(60).mean()
    df['Gold_Trend_Direction'] = (gold_ma_short > gold_ma_long).astype(int)
    df['Gold_Trend_Strength'] = (df['Gold_Price'] - gold_ma_long) / gold_ma_long
    print("   ✓ Gold trend persistence (20d vs 60d MA)")

    # 6. GOLD SAFE HAVEN ACTIVATION - Crisis correlation (UNIQUE SIGNAL)
    # When gold rises while stocks fall (safe haven behavior)
    if 'SP500_Returns' in df.columns:
        df['Gold_Safe_Haven'] = (
            (df['Gold_Returns'] > 0) & 
            (df['SP500_Returns'] < -1.0)  # Significant stock decline
        ).astype(int)
        print("   ✓ Gold safe haven activation (crisis behavior)")

    # 7. SELECTIVE GOLD LAGS - Only the most predictive (MEMORY SIGNAL)
    # Research shows 5-day lag is often most predictive for gold
    df['Gold_Returns_lag5'] = df['Gold_Returns'].shift(5)
    print("   ✓ Gold 5-day lag (momentum persistence)")

    # Count added gold features
    gold_features = [col for col in df.columns if 'Gold' in col or 'Real_Interest_Rate' in col]
    print(f"   Total optimized gold features: {len(gold_features)}")

    # Display the selected features
    print("\n Selected Gold Features:")
    for i, feature in enumerate(gold_features, 1):
        print(f"   {i:2d}. {feature}")

    return df

In [31]:
def create_core_technical_features(df):
    print("Creating core technical features (53-feature winning set)...")

    df_enhanced = df.copy()

    # 1. VIX features (fear/volatility indicators)
    if 'VIX' in df_enhanced.columns:
        for lag in [1, 5, 10]:
            df_enhanced[f'VIX_lag_{lag}'] = df_enhanced['VIX'].shift(lag)

        df_enhanced['VIX_MA_10'] = df_enhanced['VIX'].rolling(10).mean()
        df_enhanced['VIX_relative'] = df_enhanced['VIX'] / df_enhanced['VIX_MA_10']
        df_enhanced['VIX_spike'] = (df_enhanced['VIX'] > df_enhanced['VIX_MA_10'] * 1.5).astype(int)
        print("   ✓ VIX features (6 features)")

    # 2. Rolling volatility features
    if 'SP500_Returns' in df_enhanced.columns:
        for window in [5, 10, 20]:
            df_enhanced[f'Volatility_{window}d'] = df_enhanced['SP500_Returns'].rolling(window).std()
        print("   ✓ Rolling volatility features (3 features)")

    # 3. Moving average features
    if 'SP500_Close' in df_enhanced.columns:
        for ma_period in [10, 20, 50]:
            df_enhanced[f'MA_{ma_period}'] = df_enhanced['SP500_Close'].rolling(ma_period).mean()
            df_enhanced[f'Price_to_MA_{ma_period}'] = df_enhanced['SP500_Close'] / df_enhanced[f'MA_{ma_period}']

        # MA crossover signals
        df_enhanced['MA_10_vs_20'] = (df_enhanced['MA_10'] > df_enhanced['MA_20']).astype(int)
        df_enhanced['MA_20_vs_50'] = (df_enhanced['MA_20'] > df_enhanced['MA_50']).astype(int)
        print("   ✓ Moving average features (8 features)")

    # 4. RSI and momentum oscillators
    if 'SP500_Close' in df_enhanced.columns:
        df_enhanced['RSI'] = calculate_rsi(df_enhanced['SP500_Close'])
        df_enhanced['RSI_overbought'] = (df_enhanced['RSI'] > 70).astype(int)
        df_enhanced['RSI_oversold'] = (df_enhanced['RSI'] < 30).astype(int)
        print("   ✓ RSI features (3 features)")

    # 5. Momentum features
    if 'SP500_Close' in df_enhanced.columns:
        for period in [5, 10, 20]:
            df_enhanced[f'Momentum_{period}d'] = df_enhanced['SP500_Close'].pct_change(period)
            df_enhanced[f'Momentum_{period}d_positive'] = (df_enhanced[f'Momentum_{period}d'] > 0).astype(int)
        print("   ✓ Momentum features (6 features)")

    # 6. Time-based features
    df_enhanced['DayOfWeek'] = df_enhanced.index.dayofweek
    df_enhanced['Month'] = df_enhanced.index.month
    df_enhanced['Quarter'] = df_enhanced.index.quarter
    df_enhanced['IsMonthEnd'] = df_enhanced.index.is_month_end.astype(int)
    print("   ✓ Time features (4 features)")

    # 7. Yield curve features
    if 'Yield_Spread' in df_enhanced.columns:
        df_enhanced['Yield_Spread_MA'] = df_enhanced['Yield_Spread'].rolling(20).mean()
        df_enhanced['Yield_Spread_relative'] = df_enhanced['Yield_Spread'] / df_enhanced['Yield_Spread_MA']
        df_enhanced['Yield_Curve_Inversion'] = (df_enhanced['Yield_Spread'] < 0).astype(int)
        print("   ✓ Yield curve features (3 features)")

    # 8. Treasury features
    if 'Treasury_10Y' in df_enhanced.columns:
        df_enhanced['Treasury_10Y_change'] = df_enhanced['Treasury_10Y'].diff()
        df_enhanced['Treasury_10Y_MA'] = df_enhanced['Treasury_10Y'].rolling(20).mean()
        df_enhanced['Treasury_Rising'] = (df_enhanced['Treasury_10Y_change'] > 0).astype(int)
        print("   ✓ Treasury features (3 features)")

    # 9. CPI features
    if 'CPI' in df_enhanced.columns:
        df_enhanced['CPI_YoY'] = df_enhanced['CPI'].pct_change(252)
        df_enhanced['CPI_acceleration'] = df_enhanced['CPI_YoY'].diff()
        print("   ✓ CPI features (2 features)")

    # 10. Unemployment features
    if 'Unemployment' in df_enhanced.columns:
        df_enhanced['Unemployment_change'] = df_enhanced['Unemployment'].diff()
        df_enhanced['Unemployment_Rising'] = (df_enhanced['Unemployment_change'] > 0).astype(int)
        print("   ✓ Unemployment features (2 features)")

    # 11. Cross-asset correlations
    if 'Treasury_10Y' in df_enhanced.columns and 'SP500_Returns' in df_enhanced.columns:
        df_enhanced['Stock_Bond_Corr'] = df_enhanced['SP500_Returns'].rolling(60).corr(
            df_enhanced['Treasury_10Y'].diff()
        )
        print("   ✓ Cross-asset correlation (1 feature)")

    return df_enhanced

In [40]:
def prepare_optimized_lstm_data():
    """Main data preparation with optimized gold integration"""
    print(" OPTIMIZED Data Preparation for LSTM")
    print("="*45)

    filename = 'data/financial_dataset_daily.csv'

    try:
        df = pd.read_csv(filename, index_col=0, parse_dates=True)
        print(f"Loaded {filename}: {df.shape}")

    except FileNotFoundError:
        print("FileNotFoundError")

    # Create core features (the winning 53-feature set)
    df_core = create_core_technical_features(df)

    # Add optimized gold features
    df_optimized = add_optimized_gold_features(df_core)

    # Clean data
    df_optimized = df_optimized.dropna()

    # Final feature count
    original_features = df.shape[1]
    optimized_features = df_optimized.shape[1]
    gold_features = len([col for col in df_optimized.columns if 'Gold' in col or 'Real_Interest_Rate' in col])
    core_features = optimized_features - gold_features

    print(f"\n OPTIMIZED DATA PREPARATION COMPLETE!")
    print("="*45)
    print(f"Original features: {original_features}")
    print(f"Core features: {core_features}")
    print(f"Optimized gold features: {gold_features}")
    print(f"Total optimized features: {optimized_features}")
    print(f"Final observations: {len(df_optimized)}")

    # Feature breakdown
    print(f"\n FEATURE BREAKDOWN:")
    feature_categories = {
        'Market Data': [col for col in df_optimized.columns if 'SP500' in col],
        'VIX Features': [col for col in df_optimized.columns if 'VIX' in col],
        'Technical Indicators': [col for col in df_optimized.columns if any(x in col for x in ['MA_', 'RSI', 'Momentum']) and 'Gold' not in col],
        'Volatility': [col for col in df_optimized.columns if 'Volatility' in col],
        'Time Features': [col for col in df_optimized.columns if any(x in col for x in ['DayOfWeek', 'Month', 'Quarter'])],
        'Macro Economic': [col for col in df_optimized.columns if any(x in col for x in ['Treasury', 'Yield', 'CPI', 'Unemployment']) and 'Gold' not in col],
        'Optimized Gold': [col for col in df_optimized.columns if 'Gold' in col or 'Real_Interest_Rate' in col]
    }

    for category, features in feature_categories.items():
        if features:
            print(f"\n{category} ({len(features)} features):")
            for feature in features:
                print(f"   • {feature}")

    # Save optimized dataset
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'data/optimized_financial_data_{timestamp}.csv'
    df_optimized.to_csv(filename)

    # Standard filename for easy loading
    df_optimized.to_csv('data/optimized_financial_data.csv')

    print(f"\n Optimized data saved:")
    print(f"   Main file: optimized_financial_data.csv")
    print(f"   Timestamped: {filename}")

    return df_optimized

In [41]:
if __name__ == "__main__":
    df_optimized = prepare_optimized_lstm_data()

 OPTIMIZED Data Preparation for LSTM
Loaded data/financial_dataset_daily.csv: (8918, 14)
Creating core technical features (53-feature winning set)...
   ✓ VIX features (6 features)
   ✓ Rolling volatility features (3 features)
   ✓ Moving average features (8 features)
   ✓ RSI features (3 features)
   ✓ Momentum features (6 features)
   ✓ Time features (4 features)
   ✓ Yield curve features (3 features)
   ✓ Treasury features (3 features)
   ✓ CPI features (2 features)
   ✓ Unemployment features (2 features)
   ✓ Cross-asset correlation (1 feature)
 Adding optimized gold features (quality over quantity)...
   Attempting gold futures (GC=F)...
   Gold futures data: 6298 observations
   ✓ Basic gold price and returns
   ✓ Gold/SPX ratio (primary risk sentiment indicator)
   ✓ Gold vs real interest rates (monetary policy signal)
   ✓ Gold momentum (10-day, different from stock momentum)
   ✓ Gold volatility regime (crisis detection)
   ✓ Gold trend persistence (20d vs 60d MA)
   ✓ Gold sa